# Importing Libraries

In [1]:
import torch
#from torch import nn
from torchvision import datasets, transforms

import numpy as np

#from sklearn import datasets

import matplotlib as mpl
#from matplotlib.colors import LinearSegmentedColormap
#from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt

#import reprlib
import sys

# Configuring Visualization Parameters

In [2]:
%matplotlib inline

In [3]:
XINHUI = "#7a7374"
XUEBAI = "#fffef9"
YINBAI = "#f1f0ed"
YINHUI = "#918072"

figure_size = (16, 9)

In [4]:
custom_params = {
    "axes.axisbelow": True,
    "axes.edgecolor": YINBAI,
    "axes.facecolor": XUEBAI,
    "axes.grid": True,
    "axes.labelcolor": XINHUI,
    "axes.spines.right": False,
    "axes.spines.top": False,
    "axes.titlecolor": XINHUI,
    "figure.edgecolor": YINBAI,
    "figure.facecolor": XUEBAI,
    "grid.alpha": .8,
    "grid.color": YINBAI,
    "grid.linestyle": "--",
    "grid.linewidth": 1.2,
    "legend.edgecolor": YINHUI,
    "patch.edgecolor": XUEBAI,
    "patch.force_edgecolor": True,
    "text.color": XINHUI,
    "xtick.color": YINHUI,
    "ytick.color": YINHUI,
}

mpl.rcParams.update(custom_params)

# Configuring Other Notebook Parameters

In [5]:
#reprlib_rules = reprlib.Repr()
#reprlib_rules.maxother = 250

# Pre-installing Custom Functions

In [6]:
sys.path.append("../")

In [7]:
from Modules import *

# Practicing in Stages

## Image Transforms

In [8]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, ), (0.5, ))])

training_dataset = datasets.MNIST(root="../Datasets",
                                  train=True,
                                  download=True,
                                  transform=transform)

tabulation = Form_Generator()
font_colors_list = tabulation.color_selector()

tabulation.heading_printer("Loading and transformation of the MNIST dataset")

statements = [
    """
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, ), (0.5, ))])

training_dataset = datasets.MNIST(root="../Datasets",
                                  train=True,
                                  download=True,
                                  transform=transform)
"""
]
tabulation.statement_generator(statements)

variables = ["transform", "training_dataset"]
values = [str(transform), str(training_dataset)]
tabulation.variable_generator(variables, values)

Loading and transformation of the MNIST dataset

    +--------------------------------------------------------+
    | Statement                                              |
    +--------------------------------------------------------+
    | transform = transforms.Compose(                        |
    |     [transforms.ToTensor(),                            |
    |      transforms.Normalize((0.5, ), (0.5, ))])          |
    |                                                        |
    | training_dataset = datasets.MNIST(root="../Datasets",  |
    |                                   train=True,          |
    |                                   download=True,       |
    |                                   transform=transform) |
    +--------------------------------------------------------+
    +------------------+----------------------------------------+
    | Variable         | Value                                  |
    +------------------+---------------------------------------

In [9]:
training_loader = torch.utils.data.DataLoader(dataset=training_dataset,
                                              batch_size=100,
                                              shuffle=True)

tabulation = Form_Generator()
tabulation.heading_printer("Configuration of the training batches")

statements = [
    """
training_loader = torch.utils.data.DataLoader(dataset=training_dataset,
                                              batch_size=100,
                                              shuffle=True)
"""
]
tabulation.long_statement_generator(statements)

expressions = [
    "len(training_loader)", "list(training_loader)[0][0].shape",
    "list(training_loader)[0][1].shape",
    "next(iter(training_loader))[0].shape",
    "next(iter(training_loader))[1].shape"
]
results = [
    str(len(training_loader)),
    str(list(training_loader)[0][0].shape),
    str(list(training_loader)[0][1].shape),
    str(next(iter(training_loader))[0].shape),
    str(next(iter(training_loader))[1].shape),
]
tabulation.expression_generator(expressions, results, 12)

Configuration of the training batches

    +-----------------------------------------------------------+
    | Statement                                                 |
    +-----------------------------------------------------------+
    | training_loader =                                         |
    |     torch.utils.data.DataLoader(dataset=training_dataset, |
    |     batch_size=100,                                       |
    |     shuffle=True)                                         |
    +-----------------------------------------------------------+
    +--------------------------------------+----------------------+
    | Expression                           | Result               |
    +--------------------------------------+----------------------+
    | len(training_loader)                 | 600                  |
    | list(training_loader)[0][0].shape    | torch.Size([100, 1,  |
    |                                      |             28, 28]) |
    | list(training_loade

In [10]:
def im_convert(tensor):
    image = tensor.clone().detach().numpy()
    image = image.transpose(1, 2, 0)
    image = image * np.array((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    image = image.clip(0, 1)
    return image


tabulation = Form_Generator()
tabulation.heading_printer("Function definition for plotting tensor images")

definitions = [
    """
def im_convert(tensor):
    image = tensor.clone().detach().numpy()
    image = image.transpose(1, 2, 0)
    image = image * np.array((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    image = image.clip(0, 1)
    return image
"""
]
tabulation.definition_generator(definitions)

Function definition for plotting tensor images

    +----------------------------------------------------------+
    | Definition                                               |
    +----------------------------------------------------------+
    | def im_convert(tensor):                                  |
    |     image = tensor.clone().detach().numpy()              |
    |     image = image.transpose(1, 2, 0)                     |
    |     image = image * np.array((0.5, 0.5, 0.5), (0.5, 0.5, |
    |     0.5))                                                |
    |     image = image.clip(0, 1)                             |
    |     return image                                         |
    +----------------------------------------------------------+


In [11]:
dataiter = iter(training_loader)
images, labels = next(dataiter)

In [12]:
next(dataiter)[0].shape

next(dataiter)[1]

tensor([0, 0, 3, 2, 4, 0, 8, 3, 7, 2, 3, 4, 3, 2, 5, 1, 2, 5, 6, 4, 6, 8, 7, 3,
        3, 8, 6, 9, 1, 3, 4, 2, 7, 0, 7, 4, 2, 0, 1, 0, 4, 8, 6, 2, 0, 7, 1, 1,
        5, 5, 1, 6, 8, 4, 0, 3, 1, 6, 3, 6, 1, 8, 9, 3, 0, 9, 0, 6, 5, 8, 0, 5,
        5, 2, 6, 0, 1, 3, 0, 8, 0, 6, 8, 4, 4, 1, 3, 2, 9, 3, 9, 8, 8, 4, 2, 0,
        2, 2, 0, 3])

In [13]:
a = statements[0].strip().split("\n")

In [14]:
for i in statements:
    i = i.strip().split("\n")
    map(lambda i: i.strip(), i)
    print(i)

['training_loader = torch.utils.data.DataLoader(dataset=training_dataset,', '                                              batch_size=100,', '                                              shuffle=True)']


In [15]:
a

['training_loader = torch.utils.data.DataLoader(dataset=training_dataset,',
 '                                              batch_size=100,',
 '                                              shuffle=True)']

In [16]:
list(map(lambda i: i.strip(), a))

['training_loader = torch.utils.data.DataLoader(dataset=training_dataset,',
 'batch_size=100,',
 'shuffle=True)']